### Setup
#### Import Necessary Packages and suppress noisy warnings:

In [9]:
import pandas
import warnings
import yfinance

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

#### Define a default dataframe grid style:

In [10]:
def style_basic(styler):

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Remove index column
    styler.hide(axis='index')

    return styler

#### Define Script PARAMS

In [11]:
# Stock Basket
stocks = ['AAPL', 'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MS', 'TRV', 'GOOG', 'TSLA']

# Analysis Window - 5 Years (Correspond to Risk Free Rate)
start_date = '2019-04-01'
end_date = '2024-04-01'

# Risk-Free Rate - 5 Year
rf = 0.047

# Investment Budget
budget = 250000

### Extract Historical Data & Calculate Metrics
#### Define a function to pull historial data from Yahoo Finance and calculate metrics

In [12]:
def calculate_metrics(ticker, start_date, end_date):
    # Fetch historical data using yfinance
    stock_data = yfinance.download(ticker, start=start_date, end=end_date)

    # Calculate daily returns
    stock_data['Daily_Return'] = stock_data['Adj Close'].pct_change()

    # Calculate volatility (annualized standard deviation of daily returns)
    volatility = stock_data['Daily_Return'].std() * (252 ** 0.5)

    # Calculate cumulative returns
    cumulative_returns = (1 + stock_data['Daily_Return']).cumprod() - 1

    # Calculate drawdowns
    cumulative_max = cumulative_returns.cummax()
    drawdowns = (cumulative_returns - cumulative_max) / (cumulative_max + 1)

    # Calculate metrics
    total_return = cumulative_returns.iloc[-1]
    max_drawdown = drawdowns.min()
    avg_annual_return = cumulative_returns.mean() * 252
    sharpe_ratio = avg_annual_return / volatility

    return {
        'Ticker': ticker,
        'Total Return': total_return,
        'Volatility': volatility,
        'Max Drawdown': max_drawdown,
        'Average Annual Return': avg_annual_return,
        'Sharpe Ratio': sharpe_ratio
    }

#### Apply function to our basket of stocks

In [13]:
metrics_data = []
for ticker in stocks:
    metrics = calculate_metrics(ticker, start_date, end_date)
    metrics_data.append(metrics)

# Create a DataFrame to display results
metrics_df = pandas.DataFrame(metrics_data)
metrics_df

# Style
(metrics_df.style.pipe(style_basic)
    .set_caption('Historical Performance Metrics')
    .set_table_styles(
    [{
        'selector': 'th.col_heading',
        'props': 'text-align: center'
    }, {
        'selector': 'caption',
        'props': [('text-align', 'center'),
                  ('font-size', '14pt')]}
    ]))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Ticker,Total Return,Volatility,Max Drawdown,Average Annual Return,Sharpe Ratio
AAPL,2.725669,0.317720,-0.314273,450.532552,1418.017736
GS,1.398248,0.324744,-0.456210,154.707969,476.399142
IBM,0.774494,0.264370,-0.389801,26.550501,100.429213
INTC,-0.074048,0.393424,-0.616262,-22.860307,-58.106002
JNJ,0.304422,0.198076,-0.273663,53.131438,268.237819
JPM,1.223350,0.317913,-0.436265,97.672254,307.229282
MS,1.531084,0.352370,-0.513258,211.534523,600.318988
TRV,0.870258,0.284992,-0.462790,50.355630,176.691185
GOOG,1.549500,0.319614,-0.446018,182.669296,571.530805
TSLA,8.118368,0.646539,-0.736322,2094.521330,3239.590956


#### Create Mean-Variance Optimized Portfolio and generate weightage

In [14]:
# Fetch historical data using yfinance
stocks_info = yfinance.download(stocks, start=start_date)["Close"]

# Calculate Expected annualized returns
mu = expected_returns.mean_historical_return(df)
# Calculate annualized sample covariance matrix returns
S = risk_models.sample_cov(stocks_info)
 
# Generate the Efficient Frontier object - optimize for the MAX sharpe ratio - Based on Risk Free Rate
efficient_frontier = EfficientFrontier(mu, S)
weights = efficient_frontier.max_sharpe(rf) ##Defaults to 0.02 if 'rf' not specified

# Round small marginal weights to 0
cleaned_weights = efficient_frontier.clean_weights()

# Create a DataFrame to display results
cleaned_weights_df = pandas.DataFrame([cleaned_weights])
cleaned_weights_df

# Style
(cleaned_weights_df.style.pipe(style_basic)
    .set_caption('Optimized Portfolio Weightage')
    .set_table_styles(
    [{
        'selector': 'th.col_heading',
        'props': 'text-align: center'
    }, {
        'selector': 'caption',
        'props': [('text-align', 'center'),
                  ('font-size', '14pt'),
                  ('font-weight', 'bold')]}
    ]))

[*********************100%%**********************]  10 of 10 completed


NameError: name 'df' is not defined

#### Display Optimized Portfolio Performance Metrics

In [7]:
efficient_frontier.portfolio_performance(verbose=True, risk_free_rate=rf)

NameError: name 'efficient_frontier' is not defined

#### Display Discrete Stock share Allocations based on Optimal Portfolio

In [81]:
prices = get_latest_prices(df)
allocations_info = DiscreteAllocation(cleaned_weights, prices, total_portfolio_value=budget)

# Allocate using 'Linear Programming'
#allocation, leftover = allocations_info.lp_portfolio()

# Allocate using 'Greedy Portfolio'
allocation, leftover = allocations_info.greedy_portfolio()


allocation_df = pandas.DataFrame([allocation])
allocation_df

# Style
(allocation_df.style.pipe(style_basic)
    .set_caption('Discrete Optimal Stock Allocation')
    .set_table_styles(
    [{
        'selector': 'th.col_heading',
        'props': 'text-align: center'
    }, {
        'selector': 'caption',
        'props': [('text-align', 'center'),
                  ('font-size', '16pt'),
                  ('font-weight', 'bold')]}
    ]))


AAPL,TSLA,GOOG
894,313,264


In [82]:
print("Funds Remaining after optimal allocation:", "$", leftover)

Funds Remaining after optimal allocation: $ 116.86872863769531
